In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements, solve_ritz_two_vec
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho, coeff_gram_schmidt
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
from functools import reduce


In [2]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]


#load Hamiltonian generated by above coefficients
H_correct = scipy.io.mmread('../data_water/H_water_correct.mtx').tocsr()
H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# e, v = sparse.linalg.eigsh(H_correct_10e, which = 'SA', k = 10)
# e_ground = e[0]
# e_1st_ex = e[1]

e_ground = -84.92262983120877
e_1st_ex = -84.52780562388426
e_2nd_ex = -84.46820215626565
e_3rd_ex = -84.42486181064207
e_4th_ex = -84.42379006099287

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


/home/ge49cag/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [3]:
#excited states for different molecules could have different total spins, please try different spin sections.
#for water in sto-6g: 1st excited state has spin 1
HFS = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])
HFS2 = generate_single_state(7, [3, 3, 3, 3, 1, 2, 0])
HFS3 = generate_single_state(7, [3, 3, 3, 3, 1, 0, 2])
HFS4 = generate_single_state(7, [3, 3, 3, 3, 0, 1, 2])

initial = add_mps(HFS4, add_mps(HFS3, add_mps(HFS, HFS2)))
initial.orthonormalize('right')
initial = initial.as_vector()

In [4]:
N = 30

Krylov_vector_list = []

psi_kerylov_start = copy.deepcopy(initial)
#psi_kerylov_start = copy.deepcopy(psi_mps1)
#psi_kerylov_start = copy.deepcopy(psi_mps10)
#psi_mps10 

psi_kerylov_start /= norm(psi_kerylov_start)
Krylov_vector_list.append(psi_kerylov_start)



for i in range (1, N):
    #krylov_vector = H_correct_10 @ Krylov_vector_list[i-1]
    krylov_vector = H_correct @ Krylov_vector_list[i-1]
    krylov_vector /= norm(krylov_vector)
    #temp = copy.deepcopy(krylov_vector)
    if i == 1:
        krylov_vector +=  -np.vdot(Krylov_vector_list[0], krylov_vector)*Krylov_vector_list[0]
    else:
        for j in range (2):
            #print(i+j-2)
            krylov_vector +=  -np.vdot(Krylov_vector_list[i+j-2], krylov_vector)*Krylov_vector_list[i+j-2]
    krylov_vector /= norm(krylov_vector)
    Krylov_vector_list.append(krylov_vector)

In [5]:
# for i in range (N):
#     for j in range (N):
#         if i !=j :
#             print(i, j, np.vdot(Krylov_vector_list[i], Krylov_vector_list[j]))

In [6]:
TN = np.zeros([len(Krylov_vector_list),len(Krylov_vector_list)])
for i in range (TN.shape[0]):
    for j in range (TN.shape[1]):
        if abs(i - j) < 2:
            #TN[i, j] = np.vdot(Krylov_vector_list[i], H_correct_10@Krylov_vector_list[j])
            TN[i, j] = np.vdot(Krylov_vector_list[i], H_correct@Krylov_vector_list[j])
      
print(TN.shape)

e_krylov, v_krylov = np.linalg.eigh(TN)
print(e_krylov[0] - e_ground)
#print(e_krylov[1] - e_1st_ex)

(30, 30)
1.2040741395935584e-05


In [7]:
shift = e_krylov[0]
Q, R = np.linalg.qr(TN - shift*np.identity(TN.shape[0]))

Krylov_vector_list_restart_1 = []
for i in range (len(Krylov_vector_list)):
    
    temp = np.zeros_like(Krylov_vector_list[0])
    for a, v in zip(Q[:,i], Krylov_vector_list):
        temp += a * v
    Krylov_vector_list_restart_1.append(temp)
    
#Krylov_vector_list_restart_1.pop(-1)
    

In [8]:
np.vdot(Krylov_vector_list_restart_1[-1], H_correct@Krylov_vector_list_restart_1[-1]) - e_ground

1.2040729785667281e-05

In [18]:
TN_restart_1 = np.zeros([len(Krylov_vector_list_restart_1),len(Krylov_vector_list_restart_1)])
for i in range (TN_restart_1.shape[0]):
    for j in range (TN_restart_1.shape[1]):
        #if abs(i - j) < 2:
            #TN[i, j] = np.vdot(Krylov_vector_list[i], H_correct_10@Krylov_vector_list[j])
        TN_restart_1[i, j] = np.vdot(Krylov_vector_list_restart_1[i], H_correct@Krylov_vector_list_restart_1[j])
      
print(TN_restart_1.shape)

e_krylov_restart_1, v_krylov_restart_1 = np.linalg.eigh(TN_restart_1)
print(e_krylov_restart_1[0] - e_ground)
#print(e_krylov_restart_1[0] - e_1st_ex)

(40, 40)
6.314071754331962e-08


In [10]:
for i in range (len(Krylov_vector_list_restart_1), len(Krylov_vector_list_restart_1) + 10, 1):
    krylov_vector = H_correct @ Krylov_vector_list_restart_1[i - 1]
    krylov_vector /= norm(krylov_vector)
    # for j in range (2):
    #     krylov_vector +=  -np.vdot(Krylov_vector_list_restart_1[i+j-2], krylov_vector)*Krylov_vector_list_restart_1[i+j-2]
    for j in range (len(Krylov_vector_list_restart_1)):
        krylov_vector +=  -np.vdot(Krylov_vector_list_restart_1[j], krylov_vector)*Krylov_vector_list_restart_1[j]
    krylov_vector /= norm(krylov_vector)
    Krylov_vector_list_restart_1.append(krylov_vector)
    

In [11]:
# for i in range (len(Krylov_vector_list_restart_1)):
#     for j in range (len(Krylov_vector_list_restart_1)):
#         if i !=j :
#             print(i, j, np.vdot(Krylov_vector_list_restart_1[i], Krylov_vector_list_restart_1[j]))

In [12]:
def gram_schmidt(A):
    """Orthogonalize a set of vectors stored as list A."""
    # Get the number of vectors.
    n = len(A)
    for j in range(n):
        # To orthogonalize the vector in vec j with respect to the
        # previous vectors, subtract from it its projection onto
        # each of the previous vectors.
        for k in range(j):
            A[j] -= np.dot(A[k], A[j]) * A[k]
        A[j] = A[j] / np.linalg.norm(A[j])
    return A

In [13]:
Krylov_vector_list_restart_1_reortho = gram_schmidt(Krylov_vector_list_restart_1)

In [14]:
# for i in range (len(Krylov_vector_list_restart_1_reortho)):
#     for j in range (len(Krylov_vector_list_restart_1_reortho)):
#         if i !=j :
#             print(i, j, np.vdot(Krylov_vector_list_restart_1_reortho[i], Krylov_vector_list_restart_1_reortho[j]))

In [15]:
TN_restart_1 = np.zeros([len(Krylov_vector_list_restart_1_reortho),len(Krylov_vector_list_restart_1_reortho)])
for i in range (TN_restart_1.shape[0]):
    for j in range (TN_restart_1.shape[1]):
        if abs(i - j) < 2:
            #TN[i, j] = np.vdot(Krylov_vector_list[i], H_correct_10@Krylov_vector_list[j])
            TN_restart_1[i, j] = np.vdot(Krylov_vector_list_restart_1_reortho[i], H_correct@Krylov_vector_list_restart_1_reortho[j])
      
print(TN_restart_1.shape)

e_krylov_restart_1, v_krylov_restart_1 = np.linalg.eigh(TN_restart_1)
print(e_krylov_restart_1[0] - e_1st_ex)
print(e_krylov_restart_1[1] - e_1st_ex)

(40, 40)
-0.3948194822818607
0.013143669302507988


In [16]:
TN_restart_1.shape

(40, 40)

In [17]:
1.2533177425666508e-05

1.2533177425666508e-05